In [296]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from   sklearn.model_selection import train_test_split
from   sklearn.metrics import accuracy_score
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F



In [280]:
X_train = pd.read_csv('train_ml2_2021.csv')
y_train = X_train['target'].copy()
X_train.drop(['target'], inplace=True, axis=1)

X_test = pd.read_csv('test0.csv')
y_test = X_test['target'].copy()
X_test.drop(['obs_id','target'], inplace=True, axis=1)

X_train.columns

Index(['problem_id', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8',
       ...
       'v968', 'v969', 'v970', 'v971', 'v972', 'v973', 'v974', 'v975', 'v976',
       'v977'],
      dtype='object', length=979)

In [292]:
X_train.groupby('problem_id').count()['v0'].quantile(0.75)

408.0

In [3]:
y_train.value_counts()

0    4124
1    3175
2     729
3     255
4      19
Name: target, dtype: int64

In [4]:
X_train = torch.FloatTensor(np.array(X_train))
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(np.array(X_test))
y_test = torch.LongTensor(np.array(y_test))

In [5]:
X_train.shape

torch.Size([8302, 979])

In [6]:
y_train.unique(), y_test.unique()

(tensor([0, 1, 2, 3, 4]), tensor([0]))

In [7]:
# for col in df_train.columns:
#     data = df_train[col]
#     title = f"EDA of {col}"
    
#     fig, ax = plt.subplots(figsize=(8, 8))
    
#     ax.hist(data)
#     plt.title(title)
#     plt.show()

# Oh wow lots of features. Looks like the majority are already scaled between
# 0 and 1, so we should be able to get away with scaling it to a normal distributions

len(X_train.columns)

AttributeError: 'Tensor' object has no attribute 'columns'

In [ ]:
X_train.dtypes.unique()

# Baseline

In [ ]:
from   sklearn.decomposition   import PCA
from   sklearn.model_selection import *
from   sklearn.model_selection import train_test_split
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import *
from   sklearn.impute          import *
from   sklearn.linear_model    import *
from   sklearn.compose         import *
from   sklearn.ensemble        import *
from   sklearn.metrics         import accuracy_score

In [ ]:
model = RandomForestClassifier(n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
y_hat=model.predict(X_test)

In [ ]:
metric = accuracy_score(y_test, y_hat)

In [ ]:
metric

In [ ]:
X

In [ ]:
X = pd.read_csv('train_ml2_2021.csv')
y = X['target'].copy()
X.drop(['target'], inplace=True, axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)

X_test = pd.read_csv('test0.csv')
y_test = X_test['target'].copy()
X_test.drop(['obs_id','target'], inplace=True, axis=1)

In [ ]:
X_train = torch.FloatTensor(np.array(X_train).copy())
y_train = torch.LongTensor(np.array(y_train).copy())
X_valid =  torch.FloatTensor(np.array(X_valid).copy())
y_valid =  torch.LongTensor(np.array(y_valid).copy())

X_test =  torch.FloatTensor(np.array(X_test).copy())

# Working on a Real Model

# Model #1: Basic 2 Layer Neural Network

In [ ]:
# class Neural_Network(nn.Module):
#     def __init__(self, input_dim, hidden_dim):
#         super(MultiLogisticRegression, self).__init__()
#         ## CODE HERE
#         self.hidden = nn.Linear(input_dim, 20)
#         self.output = nn.Linear(20, 5)
        
#         self.sigmoid = nn.Sigmoid()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         ## CODE HERE
#         x = self.hidden(x)
#         x = self.sigmoid(x)
#         x = self.output(x)
#         return x

    
def neural_network(hidden_dim, input_dim=979, num_classes=5):
    net = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                        nn.ReLU(),
                        nn.Linear(hidden_dim, num_classes))
    return net #.cuda()

def train_epochs(model, x_train, y_train, x_valid, y_valid, epochs, lr=0.01, wd=1e-4):
    ## get an optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    ## convert your training data to pytorch tensors
    x = torch.FloatTensor(x_train)
    y = torch.LongTensor(y_train)

    for i in range(epochs):
        model.train()
        ## evaluate your training data to get y_hat
        y_hat = model(x)
        ## compute your loss
        loss = F.cross_entropy(y_hat, y)
        ## zero_grad
        optimizer.zero_grad()
        ## compute gradients
        loss.backward()
        ## call gradient descent
        optimizer.step()
        val_loss, val_acc, _, _ = valid_metrics(model, x_valid, y_valid)

        if i%5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % 
                  (loss.item(), val_loss, val_acc))
            
def valid_metrics(model, x_valid, y_valid):
    model.eval()
    x = torch.FloatTensor(x_valid)
    y = torch.LongTensor(y_valid)
    out = model(x)
#     print(out.shape)
    loss = F.cross_entropy(out, y)
#     print(test)
#     print("Predicted Values: ")
#     print(y_hat)
#     print("Actual Values")
#     print(y)
    _, y_pred = torch.max(out.data, 1) # 
    val_acc = y_pred.eq(y.data).sum() / y.size(0)
    return loss.item(), val_acc.item(), out, y_pred

def save_torch_to_csv(y_idx, sub_num):
    output = pd.DataFrame(y_idx)
    output.rename(columns={0:'target'}, inplace=True)
    output.index.name = 'obs_id'
    output.to_csv(f'submission{sub_num}.csv')

In [ ]:
model = neural_network(hidden_dim=100, input_dim = X_train.shape[1])
train_epochs(model, X_train, y_train, X_valid, y_valid, epochs=100, lr=.001)

## Applying the model to the Validation Set:

In [ ]:
loss, acc, out, y_pred = valid_metrics(model, X_valid, y_valid)
loss, acc, y_pred.unique()
print(f"test loss: {loss:.3f} test acc: {acc:.3f}    unique predicted values:{[int(i) for i in y_pred.unique()]}")

# Model #2: Two layer neural network with Batch Training

## Create the DataLoaders from .csv files

In [142]:
from torch.utils.data import Dataset, DataLoader

# Load in and split the data into train/test/split
X = pd.read_csv('train_ml2_2021.csv')
y = X['target'].copy()
X.drop(['target'], inplace=True, axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=X['problem_id'])
X_train['label'] = y_train.copy()
X_valid['label'] = y_valid.copy()

# Save the split as a single csv so that MyDataset can read it in
X_train.to_csv('train.csv')
X_valid.to_csv('valid.csv')

# Test set
X_test = pd.read_csv('test0.csv')
X_test.drop(['obs_id','target'], inplace=True, axis=1)



# credit: https://www.reddit.com/r/pytorch/comments/f2c39s/how_to_create_a_data_loader_from_csv_file/
class MyDataset(Dataset):
    def __init__(self, root, n_features):
        self.df = pd.read_csv(root, index_col='Unnamed: 0')
        self.data = self.df.to_numpy()
        self.x , self.y = (torch.from_numpy(self.data[:,:n_features]),
                           torch.from_numpy(self.data[:,n_features:]))
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx,:]
    def __len__(self):
        return len(self.data)
    
train = MyDataset('train.csv', 979)
train_dl = DataLoader(train, batch_size=200, shuffle=True)
valid = MyDataset('valid.csv', 979)
valid_dl = DataLoader(valid, batch_size=200, shuffle=True)

<ipython-input-142-25e7e5ebba75>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['label'] = y_train.copy()
<ipython-input-142-25e7e5ebba75>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid['label'] = y_valid.copy()


In [141]:
def neural_network(hidden_dim, input_dim=979, num_classes=5):
    net = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                        nn.ReLU(),
                        nn.Linear(hidden_dim, num_classes))
    return net #.cuda()

def train_model(model, train_dl, valid_dl, epochs, lr=0.01, wd=1e-4):  
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    for t in range(epochs):
        # one epoch
        model.train()
        total_loss = 0
        total = 0
        for x, y in train_dl: 
            x = x.float()
            y = y.long()
            y_hat = model(x)
            loss = F.cross_entropy(y_hat, y.squeeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        train_loss = total_loss/total
        valid_loss, valid_acc, _ = valid_metric(model, valid_dl)
        if t % 10 == 0: print("train loss %.3f valid loss %.3f valid acc %.3f" % (train_loss, valid_loss, valid_acc))
            
def valid_metric(model, valid_dl):
    model.eval()
    total_loss = 0
    total = 0
    correct = 0
    for x, y in valid_dl: 
        x = x.float()
        y = y.long()
        out = model(x)
        loss = F.cross_entropy(out, y.squeeze(1))
        #print(loss.item())
        total_loss += loss.item()*y.shape[0]
        total += y.shape[0]
        _, y_pred = torch.max(out.data, 1)
        correct += y_pred.eq(y.data).sum()
#         print(y_pred.eq(y.data).sum() / y.shape[0])
#         print()
    return total_loss/total, correct/total, y_pred

In [ ]:
model = neural_network(hidden_dim=50, input_dim = 979)
train_model(model, train_dl, valid_dl, epochs=100, lr=.001, wd=0)

## Apply Model Two to Validation Set:

In [ ]:
loss, acc, y_pred = valid_metric(model, valid_dl)
print(f"test loss: {loss:.3f} test acc: {acc:.3f}    unique predicted values:{[int(i) for i in y_pred.unique()]}")

# Model 3: Dropout Neural Network w/Batch Training 

In [140]:
def neural_network_dropout(hidden_dim, p, input_dim=979, num_classes=5):
    modules = []
    modules.append(nn.Linear(input_dim, hidden_dim))
    modules.append(nn.ReLU())
    if p > 0:
        modules.append(nn.Dropout(p))
    modules.append(nn.Linear(hidden_dim, num_classes))
    return nn.Sequential(*modules) #.cuda()

In [ ]:
model = neural_network_dropout(hidden_dim=50, p=0.5)
train_model(model, train_dl, valid_dl, epochs=100, lr=0.001, wd=0)
loss, acc, y_pred = valid_metric(model, valid_dl)
print(f"test loss: {loss:.3f} test acc: {acc:.3f}    unique predicted values:{[int(i) for i in y_pred.unique()]}")

# Tuning Hyperparameters:

In [ ]:
p_vals = np.linspace(0.01, 1, 100)
valid_acc = []
train_loss = []
valid_loss = []

for p in tqdm(p_vals):
    model = neural_network_dropout(hidden_dim=50, p=0.1)
    train_model(model, train_dl, valid_dl, epochs=100, lr=.001, wd=0)
    loss, acc, y_pred = valid_metric(model, valid_dl)
    valid_acc.append(acc)
    valid_loss.append(loss)
    loss, _, _ = valid_metric(model, train_dl)
    train_loss.append(loss)

In [ ]:
plt.style.use('ggplot')

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].plot(p_vals, train_loss, label='train')
axes[0].plot(p_vals, valid_loss, label='valid')
axes[0].set_xlabel('Dropout Rate')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[1].plot(p_vals, valid_acc, label='valid accuracy')
axes[1].legend()
axes[1].set_xlabel('Dropout Rate')
axes[1].set_ylabel('Accuracy')
plt.title('Optimal Dropout Rate', x=-0.1, y=1.05, fontsize=24)
plt.savefig('graphs/Optimal_Dropout_Rate.png')

In [ ]:
p_vals[np.argmax(valid_acc)]

- Will go with a dropout rate of 0.19, though the consistency in lower training loss shows overfitting is a major problem.


Two Fixes:

1. Number of training epochs
2. Size of hidden layer

In [ ]:
epochs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
valid_acc_ep = []
train_loss_ep = []
valid_loss_ep = []

for e in tqdm(epochs):
    model = neural_network_dropout(hidden_dim=50, p=0.19, seed=1)
    train_model(model, train_dl, valid_dl, epochs=e, lr=.001, wd=0)
    loss, acc, y_pred = valid_metric(model, valid_dl)
    valid_acc_ep.append(acc)
    valid_loss_ep.append(loss)
    loss, _, _ = valid_metric(model, train_dl)
    train_loss_ep.append(loss)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].plot(epochs, train_loss_ep, label='train')
axes[0].plot(epochs, valid_loss_ep, label='valid')
axes[0].legend()
axes[0].set_xlabel('Number Training Epochs')
axes[0].set_ylabel('Loss')
axes[1].plot(epochs, valid_acc_ep, label='valid')
axes[1].legend()
axes[1].set_xlabel('Number Training Epochs')
axes[1].set_ylabel('Accuracy')
plt.title('Optimal Epochs', x=-0.1, y=1.05, fontsize=24)
plt.savefig('graphs/Optimal_Epochs.png')

- Appears that the optimal number of training epochs is 40. 

In [ ]:
hd_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]
valid_acc_hd = []
train_loss_hd = []
valid_loss_hd = []

for hd in tqdm(hd_list):
    model = neural_network_dropout(hidden_dim=hd, p=0.19)
    train_model(model, train_dl, valid_dl, epochs=40, lr=.001, wd=0)
    loss, acc, y_pred = valid_metric(model, valid_dl)
    valid_acc_hd.append(acc)
    valid_loss_hd.append(loss)
    loss, _, _ = valid_metric(model, train_dl)
    train_loss_hd.append(loss)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].plot(hd_list, train_loss_hd, label='train')
axes[0].plot(hd_list, valid_loss_hd, label='valid')
axes[0].legend()
axes[0].set_xlabel('Hidden Layer Size')
axes[0].set_ylabel('Loss')
axes[1].plot(hd_list, valid_acc_hd, label='valid')
axes[1].legend()
axes[1].set_xlabel('Hidden Layer Size')
axes[1].set_ylabel('Accuracy')
plt.title('Optimal Hidden Layer Size', x=-0.1, y=1.05, fontsize=24)
plt.savefig('graphs/Optimal_Hidden_Layer_Size.png')

In [ ]:
hd_list[np.argmax(valid_acc_hd)]

- Looks like the optimal hidden layer size is 20. This is good news, as it closes the gap between the training and validation loss. Lowering the bias a little bit at a time...

In [ ]:
lr_list = [.1, .05, .01, .005, .001, .0005, .0001, .00001]
valid_acc_lr = []
train_loss_lr = []
valid_loss_lr = []

for lr in tqdm(lr_list):
    model = neural_network_dropout(hidden_dim=20, p=0.19)
    train_model(model, train_dl, valid_dl, epochs=40, lr=lr, wd=0)
    loss, acc, y_pred = valid_metric(model, valid_dl)
    valid_acc_lr.append(acc)
    valid_loss_lr.append(loss)
    loss, _, _ = valid_metric(model, train_dl)
    train_loss_lr.append(loss)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].plot(lr_list, train_loss_lr, label='train')
axes[0].plot(lr_list, valid_loss_lr, label='valid')
axes[0].legend()
axes[0].set_xlabel('Learning Rate')
axes[0].set_ylabel('Loss')
axes[1].plot(lr_list, valid_acc_lr, label='valid')
axes[1].legend()
axes[1].set_xlabel('Learning Rate')
axes[1].set_ylabel('Accuracy')
plt.title('Optimal Learning Rate', x=-0.1, y=1.05, fontsize=24)
plt.savefig('graphs/Optimal_Learning_Rate.png')

In [ ]:
list(zip(train_loss_lr, valid_loss_lr))

In [ ]:
lr_list

In [ ]:
valid_acc_lr[7]

- training and accuracy loss seemed to be the most even for at a learning rate of $10^{-4}$

# Final Model:

In [ ]:
model = neural_network_dropout(hidden_dim=20, p=0.19)
train_model(model, train_dl, valid_dl, epochs=40, lr=0.0001, wd=0)
loss, acc, y_pred = valid_metric(model, valid_dl)
print(f"test loss: {loss:.3f} test acc: {acc:.3f}    unique predicted values:{[int(i) for i in y_pred.unique()]}")

In [ ]:
X_test = pd.read_csv('test0.csv')
X_test.drop(['obs_id','target'], inplace=True, axis=1)
X_test = torch.FloatTensor(np.array(X_test))

out = model(X_test)

_, y_pred = torch.max(out.data, 1)

save_torch_to_csv(y_pred, 4)

train_loss_lr

In [ ]:
p_vals = np.linspace(0.01, 1, 100)
valid_acc = []
train_loss = []
valid_loss = []

for p in tqdm(p_vals):
    model = neural_network_dropout(hidden_dim=50, p=0.1)
    train_model(model, train_dl, valid_dl, epochs=100, lr=.001, wd=0)
    loss, acc, y_pred = valid_metric(model, valid_dl)
    valid_acc.append(acc)
    valid_loss.append(loss)
    loss, _, _ = valid_metric(model, train_dl)
    train_loss.append(loss)

In [ ]:
model = neural_network_dropout(hidden_dim=50, p=0.1)
train_model(model, train_dl, valid_dl, epochs=100, lr=.001, wd=0)
loss, acc, y_pred = valid_metric(model, valid_dl)
loss, acc, y_pred = valid_metric(model, valid_dl)
print(f"test loss: {loss:.3f} test acc: {acc:.3f}    unique predicted values:{[int(i) for i in y_pred.unique()]}")

## Save the predictions

In [ ]:
X_test = pd.read_csv('test0.csv')
X_test.drop(['obs_id','target'], inplace=True, axis=1)
X_test = torch.FloatTensor(np.array(X_test))

out = model(X_test)

_, y_pred = torch.max(out.data, 1)

save_torch_to_csv(y_pred, 3)

# Last Push for an Optimal Model:

One misunderstanding in the previous models is that the studies are related in some way. On further inspection/reflection of the data, this can be remedied by batching the data in a different way. Because we are graciously given the study that each observation came from in both the training and the test set, we will have the ability to train and test an ensemble of models based off the "problem_id" column.

The neural networks below will still rely on the parameters given in the above final model, though we will now have an ensemble of 21 of them at our fingertips to apply to each test set.

## Validating the Neural Network Models:

It's often a good idea to see how well each individual neural network model performs, so let's take a look at the accuracy of each model while saving each successive trained model into a list

In [295]:
df = pd.read_csv('train_ml2_2021.csv')
x_col = batch1.columns[0:-1]
y_col = batch1.columns[-1]
X_train, X_valid, y_train, y_valid = train_test_split(df[x_col], df[y_col], stratify=df[y_col])

class BatchData(Dataset):
    def __init__(self, X, y):
        self.X = X.to_numpy()
        self.Y = y.to_numpy()
        self.x , self.y = (torch.from_numpy(self.X),
                           torch.from_numpy(self.Y))
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return len(self.X)
    
def neural_network_dropout(hidden_dim, p, input_dim=979, num_classes=5):
    modules = []
    modules.append(nn.Linear(input_dim, hidden_dim))
    modules.append(nn.ReLU())
    if p > 0:
        modules.append(nn.Dropout(p))
    modules.append(nn.Linear(hidden_dim, 10))
    modules.append(nn.ReLU())
    if p > 0:
        modules.append(nn.Dropout(p))
    modules.append(nn.Linear(10, 25))
    modules.append(nn.ReLU())
    if p > 0:
        modules.append(nn.Dropout(p))
    modules.append(nn.Linear(25, 3))
    modules.append(nn.ReLU())
    if p > 0:
        modules.append(nn.Dropout(p))
    modules.append(nn.Linear(3, num_classes))
    return nn.Sequential(*modules) #.cuda()

# train = BatchData(X_train, y_train)
# train_dl = DataLoader(train, batch_size = len(X_train)//10, shuffle=True)

In [294]:
def train_model(model, train_dl, valid_dl, epochs, lr=0.01, wd=1e-4):  
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    for t in range(epochs):
        # one epoch
        model.train()
        total_loss = 0
        total = 0
        for x, y in train_dl: 
            x = x.float()
            y = y.long()
            y_hat = model(x)
            loss = F.cross_entropy(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        train_loss = total_loss/total
        valid_loss, valid_acc, _ = valid_metric(model, valid_dl)
        #if t % 10 == 0: print("train loss %.3f valid loss %.3f valid acc %.3f" % (train_loss, valid_loss, valid_acc))
            
def valid_metric(model, valid_dl):
    model.eval()
    total_loss = 0
    total = 0
    correct = 0
    y_pred_tot = []
    for x, y in valid_dl: 
        x = x.float()
        y = y.long()
        out = model(x)
        loss = F.cross_entropy(out, y)
        #print(loss.item())
        total_loss += loss.item()*y.shape[0]
        total += y.shape[0]
        _, y_pred = torch.max(out.data, 1)
        y_pred_tot.extend(y_pred)
        correct += y_pred.eq(y.data).sum()
#         print(y_pred.eq(y.data).sum() / y.shape[0])
#         print()
    return total_loss/total, correct/total, y_pred_tot

## Ensemble of Models (One per Study)

In [272]:
X_train, X_valid, y_train, y_valid = train_test_split(df[x_col], df[y_col], stratify=df[y_col])

models = []
accuracies = []
sizes = []

for i in tqdm(sorted(X_train['problem_id'].unique())):
    # Focus on information from the problem id
    X_batch = X_train[X_train['problem_id'] == i]
    y_batch = y_train[X_train['problem_id'] == i]
    X_val = X_valid[X_valid['problem_id'] == i]
    y_val = y_valid[X_valid['problem_id'] == i]
    
    train = BatchData(X_batch, y_batch)
    train_dl = DataLoader(train, batch_size = len(X_batch)//10, shuffle=True)
    valid = BatchData(X_val, y_val)
    valid_dl = DataLoader(valid, batch_size = len(X_val)//10, shuffle=False)
    
    model = neural_network_dropout(hidden_dim=40, p=0.1)
    train_model(model, train_dl, valid_dl, epochs=100, lr=.001, wd=0)
    
    models.append(model)
    _, _, y_pred = valid_metric(model, valid_dl)
    sizes.append(len(X_batch)+len(X_val))
    accuracies.append(accuracy_score(y_pred, y_val))

100%|██████████| 21/21 [00:18<00:00,  1.15it/s]


## One model for all data:

In [299]:
train = BatchData(X_train, y_train)
train_dl = DataLoader(train, batch_size = len(X_train)//10, shuffle=True)
valid = BatchData(X_valid, y_valid)
valid_dl = DataLoader(valid, batch_size = len(X_val)//10, shuffle=False)
model = neural_network_dropout(hidden_dim=40, p=0.1)
train_model(model, train_dl, valid_dl, epochs=100, lr=0.001, wd=0)

_, _, y_pred = valid_metric(model, valid_dl)

acc = accuracy_score(y_pred, y_valid)
acc

0.6016377649325626

# Results:

## Ensemble of Models

In [275]:
studiespt1 = [f'study {i}' for i in range(10)]
accpt1 = accuracies[:10]
sizespt1 = sizes[:10]

df_dict1 = {}

for study, acc, size in zip(studiespt1, accpt1, sizespt1):
    df_dict1[study] = [np.round(acc*100, 2), size]

    
studiespt2 = [f'study {i}' for i in range(11,21)]
accpt2 = accuracies[11:21]
sizespt2 = sizes[11:21]

df_dict2 = {}

for study, acc, size in zip(studiespt2, accpt2, sizespt2):
    df_dict2[study] = [np.round(acc*100, 2), size]

df_dict1

{'study 0': [71.58, 369],
 'study 1': [45.13, 408],
 'study 2': [55.77, 408],
 'study 3': [64.49, 408],
 'study 4': [95.06, 347],
 'study 5': [30.61, 369],
 'study 6': [75.53, 393],
 'study 7': [42.86, 388],
 'study 8': [48.35, 369],
 'study 9': [19.35, 352]}

In [276]:
df1 = pd.DataFrame(df_dict1)
df1.index = ['accuracy', 'study size']
df1

,study 0,study 1,study 2,study 3,study 4,study 5,study 6,study 7,study 8,study 9
accuracy,71.58,45.13,55.77,64.49,95.06,30.61,75.53,42.86,48.35,19.35
study size,369.00,408.00,408.00,408.00,347.00,369.00,393.00,388.00,369.00,352.00


In [279]:
df2 = pd.DataFrame(df_dict2)
df2.index = ['accuracy', 'study size']
df2

,study 11,study 12,study 13,study 14,study 15,study 16,study 17,study 18,study 19,study 20
accuracy,44.3,80.68,60.17,56.88,62.5,28.15,41.77,54.03,43.0,51.16
study size,310.0,320.00,547.00,408.00,408.0,547.00,352.00,469.00,469.0,408.00


## Single Model:

In [301]:
df_single = pd.DataFrame({'All Data': [np.round(acc,3), len(X_train)]})
df_single.index = ['accuracy', 'data size']
df_single

,All Data
accuracy,0.602
data size,6226.000


# Additional Practice Modeling (Scratch Work):

- XGboost training on one batch at a time

In [17]:
import xgboost as xgb
df = pd.read_csv('train_ml2_2021.csv')

In [5]:
# Split the training set into 21 batches
batches = []

for id in df['problem_id'].unique():
    batches.append(df[df['problem_id'] == id].copy())

In [9]:
batch1 = batches[0]
batch2 = batches[1]

In [35]:
x_col = batch1.columns[0:-1]
y_col = batch1.columns[-1]

In [52]:
# Split into train and validation set, making sure that each batch is with the other


In [67]:
sorted(X_train['problem_id'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [106]:
# Train sequentially, building off each other:
X_train, X_valid, y_train, y_valid = train_test_split(df[x_col], df[y_col], stratify=df[y_col])

for i in sorted(X_train['problem_id'].unique()):
    # Focus on information from the problem id
    X_batch = X_train[X_train['problem_id'] == i]
    y_batch = y_train[X_train['problem_id'] == i]
    xg_vals = xgb.DMatrix(X_batch, label=y_batch)
    params = {'objective': 'multi:softmax', 
              'num_class': len(y_train.unique()), 
              'max_depth': 9, 
              'verbosity': 0,
              'gamma' : 2,
              'eta' : 0.1,
              'booster':'dart',
              'sampling_method' : 'uniform'}
    
    if i == 0:
        model = xgb.train(params, xg_vals, 100)
        model.save_model('./xgb/model_0.model')
    else:
        model = xgb.train(params, xg_vals, 100, xgb_model=f'./xgb/model_{i-1}.model')
        model.save_model(f'./xgb/model_{i}.model')

In [131]:
# "Adaboost" somewhat inspiration: weigh the different models via their accuracy score. 
models = []
weights = []

for i in sorted(X_train['problem_id'].unique()):
    # Focus on information from the problem id
    X_batch = X_train[X_train['problem_id'] == i]
    y_batch = y_train[X_train['problem_id'] == i]
    X_val = X_valid[X_valid['problem_id'] == i]
    y_val = y_valid[X_valid['problem_id'] == i]
    
    xg_vals = xgb.DMatrix(X_batch, label=y_batch)
    params = {'objective': 'multi:softmax', 
              'num_class': len(y_train.unique()), 
              'max_depth': 9, 
              'verbosity': 0,
              'gamma' : 2,
              'eta' : 0.1,
              'booster':'dart',
              'sampling_method' : 'uniform'}
    
    model = xgb.train(params, xg_vals, 100)
    models.append(model)
    y_pred = model.predict(xgb.DMatrix(X_val, label=y_val))
    weights.append(accuracy_score(y_pred, y_val))

In [137]:
test_df = pd.read_csv('test0.csv')[test_df.columns[1:-1]]
test_df.head()


,problem_id,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,v968,v969,v970,v971,v972,v973,v974,v975,v976,v977
0,2,0.60,0.05,0.50,0.79,0.06,0.72,0.51,0.34,0.41,...,0.72,0.50,0.34,0.72,0.12,0.66,0.75,0.52,0.74,0.35
1,2,0.55,0.08,0.62,0.52,0.05,0.46,0.20,0.11,0.17,...,0.22,0.68,0.68,0.78,0.17,0.45,0.50,0.59,0.57,0.74
2,2,0.35,0.85,0.42,0.39,0.04,0.68,0.54,0.55,0.10,...,0.26,0.77,0.40,0.72,0.79,0.29,0.47,0.49,0.75,0.63
3,2,0.45,0.63,0.42,0.58,0.03,0.83,0.73,0.23,0.15,...,0.24,0.76,0.42,0.30,0.06,0.40,0.56,0.42,0.81,0.54
4,2,0.47,0.11,0.45,0.78,0.43,0.57,0.66,0.39,0.68,...,0.37,0.69,0.76,0.34,0.10,0.61,0.56,0.40,0.40,0.45


In [ ]:
for i in test

In [32]:
xg1 = xgb.DMatrix(X_train, label=y_train)
params = {'objective': 'multi:softmax', 'num_class': len(batch1[y_col].unique())}
model1 = xgb.train(params, xg1, 2)

[13:39:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [30]:
y_pred = model.predict(xgb.DMatrix(X_valid, label=y_valid))

In [31]:
accuracy_score(y_pred, y_valid)

0.6559139784946236

In [50]:
xgb.train?

Signature:
xgb.train(
    params,
    dtrain,
    num_boost_round=10,
    evals=(),
    obj=None,
    feval=None,
    maximize=None,
    early_stopping_rounds=None,
    evals_result=None,
    verbose_eval=True,
    xgb_model=None,
    callbacks=None,
)
Docstring:
Train a booster with given parameters.

Parameters
----------
params : dict
    Booster params.
dtrain : DMatrix
    Data to be trained.
num_boost_round: int
    Number of boosting iterations.
evals: list of pairs (DMatrix, string)
    List of validation sets for which metrics will evaluated during training.
    Validation metrics will help us track the performance of the model.
obj : function
    Customized objective function.
feval : function
    Customized evaluation function.
maximize : bool
    Whether to maximize feval.
early_stopping_rounds: int
    Activates early stopping. Validation metric needs to improve at least once in
    every **early_stopping_rounds** round(s) to continue training.
    Requires at least one it